In [1]:
import os
import architect

from datetime import datetime

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, String, DateTime, Integer
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine(os.environ["DB_CONN_STR"],echo=True)
Session = sessionmaker(bind=engine)

Base = declarative_base()

@architect.install('partition', type='range', subtype='date', constraint='day', column='ts', db=os.environ["DB_CONN_STR"])
class Calls(Base):
    __tablename__ = 'calls'
    id = Column(Integer, primary_key=True)
    ts = Column(DateTime)
    
Base.metadata.create_all(engine)

2019-01-21 19:22:54,321 INFO sqlalchemy.engine.base.Engine select version()
2019-01-21 19:22:54,322 INFO sqlalchemy.engine.base.Engine {}
2019-01-21 19:22:54,324 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-01-21 19:22:54,325 INFO sqlalchemy.engine.base.Engine {}
2019-01-21 19:22:54,328 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-21 19:22:54,329 INFO sqlalchemy.engine.base.Engine {}
2019-01-21 19:22:54,331 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-21 19:22:54,332 INFO sqlalchemy.engine.base.Engine {}
2019-01-21 19:22:54,334 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-01-21 19:22:54,335 INFO sqlalchemy.engine.base.Engine {}
2019-01-21 19:22:54,340 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

/Users/joshuagreenhalgh/.local/share/virtualenvs/sqlalchemy_architect-_WRA_Hx9/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
Calls.architect.partition.get_partition().prepare()

In [8]:
session = Session()
r1 = Calls(ts=datetime.strptime('2016-01-01 12:00','%Y-%m-%d %H:%M'))
r2 = Calls(ts=datetime.strptime('2016-01-02 12:00','%Y-%m-%d %H:%M'))
r3 = Calls(ts=datetime.strptime('2016-01-03 12:00','%Y-%m-%d %H:%M'))
r3 = Calls(ts=datetime.strptime('2016-01-04 12:00','%Y-%m-%d %H:%M'))
session.add(r1)
session.add(r2)
session.add(r3)
session.commit()

2019-01-21 19:24:53,357 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-21 19:24:53,362 INFO sqlalchemy.engine.base.Engine INSERT INTO calls (ts) VALUES (%(ts)s) RETURNING calls.id
2019-01-21 19:24:53,363 INFO sqlalchemy.engine.base.Engine {'ts': datetime.datetime(2016, 1, 1, 12, 0)}
2019-01-21 19:24:53,370 INFO sqlalchemy.engine.base.Engine INSERT INTO calls (ts) VALUES (%(ts)s) RETURNING calls.id
2019-01-21 19:24:53,371 INFO sqlalchemy.engine.base.Engine {'ts': datetime.datetime(2016, 1, 2, 12, 0)}
2019-01-21 19:24:53,373 INFO sqlalchemy.engine.base.Engine INSERT INTO calls (ts) VALUES (%(ts)s) RETURNING calls.id
2019-01-21 19:24:53,374 INFO sqlalchemy.engine.base.Engine {'ts': datetime.datetime(2016, 1, 4, 12, 0)}
2019-01-21 19:24:53,380 INFO sqlalchemy.engine.base.Engine COMMIT


In [9]:
sql = '''
SELECT table_schema,table_name
FROM information_schema.tables
WHERE table_name LIKE 'calls%'
ORDER BY table_schema,table_name;
'''
res = session.execute(sql)
res.fetchall()

2019-01-21 19:24:55,509 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-21 19:24:55,510 INFO sqlalchemy.engine.base.Engine 
SELECT table_schema,table_name
FROM information_schema.tables
WHERE table_name LIKE 'calls%%'
ORDER BY table_schema,table_name;

2019-01-21 19:24:55,511 INFO sqlalchemy.engine.base.Engine {}


[('public', 'calls'),
 ('public', 'calls_y2016d001'),
 ('public', 'calls_y2016d002'),
 ('public', 'calls_y2016d003'),
 ('public', 'calls_y2016d004')]

In [17]:
sql = '''
EXPLAIN 
SELECT * from calls
WHERE ts >= '2016-01-02'::timestamp AND ts < '2016-01-03'::timestamp
'''

res = session.execute(sql)
res.fetchall()

2019-01-21 19:32:31,899 INFO sqlalchemy.engine.base.Engine 
EXPLAIN 
SELECT * from calls
WHERE ts >= '2016-01-02'::timestamp AND ts < '2016-01-03'::timestamp

2019-01-21 19:32:31,901 INFO sqlalchemy.engine.base.Engine {}


[('Append  (cost=0.00..44.71 rows=11 width=12)',),
 ('  ->  Seq Scan on calls  (cost=0.00..4.06 rows=1 width=12)',),
 ("        Filter: ((ts >= '2016-01-02 00:00:00'::timestamp without time zone) AND (ts < '2016-01-03 00:00:00'::timestamp without time zone))",),
 ('  ->  Seq Scan on calls_y2016d002  (cost=0.00..40.60 rows=10 width=12)',),
 ("        Filter: ((ts >= '2016-01-02 00:00:00'::timestamp without time zone) AND (ts < '2016-01-03 00:00:00'::timestamp without time zone))",)]

In [18]:
sql = '''
EXPLAIN 
SELECT * from calls
WHERE ts >= '2016-01-01'::timestamp AND ts < '2016-01-03'::timestamp
'''

res = session.execute(sql)
res.fetchall()

2019-01-21 19:32:34,787 INFO sqlalchemy.engine.base.Engine 
EXPLAIN 
SELECT * from calls
WHERE ts >= '2016-01-01'::timestamp AND ts < '2016-01-03'::timestamp

2019-01-21 19:32:34,788 INFO sqlalchemy.engine.base.Engine {}


[('Append  (cost=0.00..85.36 rows=21 width=12)',),
 ('  ->  Seq Scan on calls  (cost=0.00..4.06 rows=1 width=12)',),
 ("        Filter: ((ts >= '2016-01-01 00:00:00'::timestamp without time zone) AND (ts < '2016-01-03 00:00:00'::timestamp without time zone))",),
 ('  ->  Seq Scan on calls_y2016d001  (cost=0.00..40.60 rows=10 width=12)',),
 ("        Filter: ((ts >= '2016-01-01 00:00:00'::timestamp without time zone) AND (ts < '2016-01-03 00:00:00'::timestamp without time zone))",),
 ('  ->  Seq Scan on calls_y2016d002  (cost=0.00..40.60 rows=10 width=12)',),
 ("        Filter: ((ts >= '2016-01-01 00:00:00'::timestamp without time zone) AND (ts < '2016-01-03 00:00:00'::timestamp without time zone))",)]